
### <center>Procesamiento Digital de Señales de Audio</center>
#### <center>Instituto de Ingeniería Eléctrica - UdelaR</center>
# Hoja de Ejercicios 3 - Curso 2022
## Análisis de Fourier de tiempo corto
### Procesamiento tiempo-frecuencia


## Pautas para el práctico
 - La realización del presente trabajo es individual.
 - Se espera la entrega de un PDF escrito en $\LaTeX$ o similar. El mismo tendrá:
     - Máximo de 12 páginas
     - Máximo de 2500 palabras
 - También se espera la entrega del código escrito, en scripts Python o en este mismo Jupyter Notebook.
 - La corrección del práctico se hará sobre lo entregado en el PDF, pero podremos apoyarnos en el razonamiento y comprensión demostrado en el código escrito. Recomendamos escribir el código de forma prolija para facilitar la comprensión presente y futura tanto de nosotros como de ustedes mismxs.
 - Los ejercicios marcados como $\blacklozenge$ son opcionales.


**Nombre de el/la estudiante:** 

### Como correr este notebook

Es posible descargarlo y correrlo localmente en su computadora

Tambien pueden correrlo en Google Colab usando el siguiente link.

<table align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/mrocamora/audio-dsp/blob/main/practicos/AudioDSP_Practico_2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Correr en Google Colab</a>
  </td>
</table>

In [2]:
# Al correr esta celda, se podrá acceder a archivos
# y carpetas en su cuenta de google drive.
# Puede ver la estructura de carpetas apretando en
# el icono de carpeta de la barra lateral izquierda.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import io, signal
from scipy.io.wavfile import read
from IPython.display import Audio

# Ejercicio 1

En este ejercicio se estudia la relación entre la transformada de Fourier de tiempo corto y la función de autocorrelación de tiempo corto. Si definimos la densidad espectral de potencia en tiempo corto de una señal ($x[n]$) en función de su transformada de Fourier en tiempo corto como

\begin{equation*}
	S_n(e^{jw}) = |X_n(e^{jw})|^2
\end{equation*}

y la función de autocorrelación de tiempo corto de la señal \(x[n]\) como

\begin{equation*}
	R_n[k] = \sum_{m=-\infty}^{\infty}w[n-m]x[m]w[n-k-m]x[m+k],
\end{equation*}

probar que si

\begin{equation*}
	X_n(e^{jw}) = \sum_{m=-\infty}^{\infty}x[m]w[n-m]e^{-jwm}
\end{equation*}

$R_n[k]$ y $S_n(e^{jw})$ son un par de transformadas, i.e.\ $S_n(e^{jw})$ es la transformada de Fourier de $R_n[k]$.


# Ejercicio 2

En este ejercicio se estudia un método de detección de frecuencia fundamental (o _pitch_) basado en la transformada de Fourier de tiempo corto. Se sugiere utilizar el archivo _LP-mem-6-a.wav_ para probarlo.

## Parte 1
El producto armónico espectral -Harmonic Product Spectrum, HPS- (ver _Pitch Detection in the Spectral Domain, pág. 623_, en  L. R. Rabiner and R. W. Schafer, _Theory and Applications of Digital Speech Processing_. Prentice Hall, 1st ed., 2011 [1]) está dado por

\begin{equation*}
	P_n(e^{jw}) = \prod_{r=1}^{K}|X_n(e^{jwr})|^2
\end{equation*}

Tomando el logaritmo se obtiene (log-Harmonic Product Spectrum, log-HPS),

\begin{equation*}
	\hat{P}_n(e^{jw}) = 2\sum_{r=1}^{K}\log|X_n(e^{jwr})|
\end{equation*}

Explique por qué el HPS puede usarse para detección de _pitch_. Asuma que la señal de audio es monofónica (una única fuente armónica). ¿Qué ventajas presenta el uso del log-HPS frente al HPS? ¿Qué ocurre con señales cuya frecuencia fundamental está ausente (e.g. filtrado pasa-altos por el canal de comunicación)?


## Parte 2

El espectro logarítmico acumulado (ó GLogS por sus siglas en inglés), se calcula como el promedio de logaritmo de la magnitud del espectro en posiciones armónicas de una frecuencia fundamental $f_0$, como

\begin{equation*}
	\rho_n(f_0) = \frac{1}{n_H}\sum_{i=1}^{n_H}\log|X_n(if_0)|
\end{equation*}

siendo $n_H$ la cantidad de armónicos de $f_0$ cuya frecuencia es menor a cierta frecuencia máxima $f_{\max}$.

Implemente un algoritmo de detección de _pitch_ que calcule el GLogS para valores de $f_0$ distribuidos de forma logarítmica entre $55Hz$ (A1) y $1046.5Hz$ (C6) con un paso de cuarto de tono, y $f_{\max} =  10000Hz$.

## Parte 3

Utilice el GLogS para obtener una representación tiempo-$f_0$, que denominaremos $f_0$-grama. Compare dicha representación con el espectrograma. Represente la frecuencia fundamental detectada y la frecuencia de referencia en el $f_0$-grama, para el archivo _LP-mem-6-a.wav_.

# Ejercicio 3

En este ejercicio se implementa la técnica de phase-vocoder y se la utiliza para generar transformaciones de la señal de audio. Se sugiere utilizar el archivo _singing\_voice.wav_ para probarla.

En la etapa de análisis se calcula la transformada de Fourier de tiempo corto, como 

\begin{equation*}
    X_{n_a^u}(e^{j\omega_k})=\sum_{m=-\infty}^{\infty}w_a[n_a^u-m]\, x[m]\, e^{-j\omega_kn}
\end{equation*}

en donde, $w_a[n]$ es la ventana de análisis, $\omega_k=\frac{2\pi}{N} k$, con \(N\) la cantidad de puntos de la DFT, y $n_a^u = u \, R_a$, con $R_a$ el hop de análisis en muestras y $u$ el índice de la trama temporal, de valor inicial 0.

En la etapa de síntesis se reconstruye la señal en el dominio del tiempo mediante la antitransformada de Fourier de cada trama temporal y el procedimiento de solapamiento y suma (overlap-add), como 

\begin{equation*}
    y[n]=\sum_{u=-\infty}^{\infty}w_s[n-n_s^u]y_u[n-n_s^u]
\end{equation*}

con

\begin{equation*}
    y_u[n]=\frac{1}{N}\sum_{k=0}^{N-1}Y_{n_s^u}(e^{j\omega_k}) \, e^{j\omega_kn}
\end{equation*}

en donde, $w_s[n]$ es la ventana de síntesis, y $n_s^u=u \, R_s$, siendo $R_s$ el hop de síntesis en muestras. Notar que $y_u[n]$ es la transformada inversa de Fourier de una trama de la STFT. Cuando no hay modificaciones entre la etapa de análisis y síntesis, $Y_{n_s^u}(e^{j\omega_k}) = X_{n_a^u}(e^{j\omega_k})$ y $R_s = R_a$. En ese caso la ventana de síntesis $w_s[n]$ es opcional, pero se hace importante si se aplican modificaciones, por ejemplo cuando $R_s \neq R_a$.  
  
## Parte 1

Implemente el análisis y la síntesis para $Y_{n_s^u}(e^{j\omega_k})=X_{n_a^u}(e^{j\omega_k})$ y $R_s=R_a$. Elija un valor de $R_a$ para tener reconstrucción perfecta con ventana de Hann y verifique experimentalmente. Justifique su elección.

## Parte 2

Realice modificaciones de la escala temporal usando $R_s \neq R_a$. En particular pruebe duplicando y reduciendo a la mitad la duración original. Analice los resultados obtenidos y los artefactos que se introducen.

## Parte 3

Para evitar la mayoría de los problemas introducidos debido a la inconsistencia de fase, se sugiere utilizar el procedimiento de desdoblamiento de fase (phase unwrapping). 

Asumiendo que existe un solo componente sinusoidal por bin de la DFT, podemos plantear las siguientes ecuaciones para estimar la fase de $Y_{n_s^u}(e^{j\omega_k})$, cuando transformamos la escala temporal utilizando un hop de síntesis $R_s \neq R_a$. 

Se calcula el incremento de fase heterodino, a partir del incremento de fase de tramas sucesivas 

\begin{equation*}
    \Delta\Phi_k^u=\angle X_{n_a^u}(e^{j\omega_k}) - \angle X_{n_a^{u-1}}(e^{j\omega_k}) - R_a \, \omega_k
\end{equation*}

Notar que el término $R_a\ \omega_k$ es el incremento de fase que cabría esperar si la frecuencia del componente sinusoidal correspondiera exactamente a la frecuencia de análisis. 

Se toma el argumento principal de $\Delta\Phi_k^u$ entre ($-\pi, \pi$), que denominamos $\Delta_p\Phi_k^u$. 

Luego se calcula la estimación de la frecuencia instantánea

\begin{equation*}
    \hat{\omega}_k[{n^u_a}] = \omega_k + \frac{1}{R_a} \, \Delta_p\Phi_k^u
\end{equation*}

Finalmente se calcula la fase de $Y_{n_s^u}(e^{j\omega_k})$ utilizando la fórmula de propagación de fase

\begin{equation*}
    \angle Y_{n_s^u}(e^{j\omega_k}) = \angle Y_{n_s^{u-1}}(e^{j\omega_k}) + R_s \, \hat{\omega}_k[{n^u_a}]
\end{equation*}

Notar que de acuerdo a la fórmula anterior se hace necesario acumular la fase de tramas sucesivas y establecer un valor para la fase inicial (se sugiere considerar $\angle Y_{n_s^{0}}(e^{j\omega_k}) = \angle X_{n_a^{0}}(e^{j\omega_k})$). Cabe señalar que es importante aplicar una ventana de suavizado $w_s[n]$ en la síntesis. Se sugiere utilizar ventanas de Hann con $w_a[n] = w_s[n]$. Tenga en cuenta que esto modifica el factor de escalamiento temporal. Para profundizar en el estudio de este procedimiento y otras consideraciones sobre la fase se recomienda [2,3].


Implemente el desdoblamiento de fase y compare los resultados con los obtenidos en la parte anterior. 

## Parte 4

Usando el phase-vocoder implemente las siguientes transformaciones de la señal de audio. Incluya en el informe los espectrogramas de las señales generadas y entregue también las señales de audio.

 - %**Escalamiento temporal (\textit{time-stretching})**: Si se reconstruye la señal de audio con un paso entre ventanas mayor al utilizado para obtener el espectrograma complejo, se obtendrá una señal más lenta que la original, y si se utiliza un paso entre ventanas menor, se obtiene una señal más rápida. ¿Se ve modificado el contenido espectral de la señal?
 - **Transposición en frecuencia (pitch-shifting)**: Se desea una señal de la misma duración que la señal original, pero alterando su contenido espectral. Para ello deben realizarse dos acciones complementarias: un escalamiento en el tiempo y un cambio en la frecuencia de muestreo. Por ejemplo, si se desea subir/bajar el contenido espectral de la señal un semitono, se debe aumentar/disminuir la duración de la señal por un factor de $2^{\frac{1}{12}}$, y luego aumentar/disminuir la frecuencia de muestreo por el mismo factor, de modo de obtener una señal de la misma duración que la original.
 - %$\blacklozenge$ **Voz robótica**: Para generar este efecto, se debe reconstruir la señal con el mismo paso entre ventanas que el utilizado para obtener el espectrograma complejo, pero descartando la información de fase (utilizando fase cero). Analizando este resultado, ¿es importante la información de fase para la reconstrucción de la señal?
 - $\blacklozenge$ **Armonizador**: Utilizando el efecto de transposición en frecuencia sumar a la señal original una versión desplazada una quinta (factor de $2^{\frac{7}{12}}\simeq\frac{3}{2}$).
 - $\blacklozenge$ **Coro (_chorus_)**: Consiste en simular que la señal de voz de un único interprete es entonada por varias voces cantando al unísono (es una variación del efecto de armonización). Se deben superponer varias señales con pequeñas modificaciones de _pitch_ respecto al de la señal original y ligeramente distinto para cada uno. El cambio no debe ser mayor que un cuarto de semitono (factor de $2^{\frac{1}{48}}$).

# Referencias

[1] L. R. Rabiner and R. W. Schafer, *Theory and Applications of Digital Speech Processing*. Prentice
Hall, 1st ed., 2011.

[2] J. Laroche and M. Dolson, “Improved phase vocoder time-scale modification of audio,” *IEEE Transac-
tions on Speech and Audio processing*, vol. 7, no. 3, pp. 323–332, 1999.

[3] A. Gotzen, N. Bernardin, and D. Arfib., “Traditional implementations of a phase-vocoder: The tricks
of the trade,” in *International Conference on Digital Audio Effects, Italy*, Dec. 2000


%\section*{Ejercicio 4}
% Pitch detection using filter bank (Rabiner ej 6.14).
% Queda comentado para una futura versión del práctico.
